In [1]:
import Fermi_Hubbard_CS as fhcs
from reverse_jwt import reverse_jwt

from qiskit import Aer
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms.minimum_eigen_solvers import VQE
from qiskit.utils import QuantumInstance
from qiskit.quantum_info import SparsePauliOp



Create linear Fermi-Hubbard model with size = 2 and convert hamiltonian into a second quantization operator

In [2]:
fhm = fhcs.linear_FHM(t=-1.0, v=0.0, u=5.0, size=2)
fhm_sqop = fhm.second_q_op()
fhm_sqop

FermionicOp({'+_0 -_2': -1.0, '-_0 +_2': 1.0, '+_0 -_0': 0.0, '+_2 -_2': 0.0, '+_1 -_3': -1.0, '-_1 +_3': 1.0, '+_1 -_1': 0.0, '+_3 -_3': 0.0, '+_0 -_0 +_1 -_1': 5.0, '+_2 -_2 +_3 -_3': 5.0}, num_spin_orbitals=4, )

Map hamiltonian by Jordan-Wigner

In [3]:
qham = JordanWignerMapper().map(fhm_sqop)
qham

PauliSumOp(SparsePauliOp(['IYZY', 'IXZX', 'YZYI', 'XZXI', 'IIII', 'IIZI', 'IIIZ', 'IIZZ', 'ZIII', 'IZII', 'ZZII'],
              coeffs=[-0.5 +0.j, -0.5 +0.j, -0.5 +0.j, -0.5 +0.j,  2.5 +0.j, -1.25+0.j,
 -1.25+0.j,  1.25+0.j, -1.25+0.j, -1.25+0.j,  1.25+0.j]), coeff=1.0)

Prepare and run VQE 

In [4]:
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

optimizer = SPSA(maxiter=50)

qi = QuantumInstance(Aer.get_backend('qasm_simulator'))
vqe = VQE(ansatz,optimizer,quantum_instance=qi)

result = vqe.compute_minimum_eigenvalue(operator=qham)
optimum_circuit = ansatz.bind_parameters(result.optimal_parameters)

C:\Users\Enes Aytemiz\AppData\Local\Temp\ipykernel_20168\2925489501.py:5: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(Aer.get_backend('qasm_simulator'))
C:\Users\Enes Aytemiz\AppData\Local\Temp\ipykernel_20168\2925489501.py:6: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz,optimizer,quantum_instance=qi)


Get density matrix by CS

In [5]:
csdm = fhcs.CS_clifford(nshadows=100, reps=1, Nqubits=qham.num_qubits,circuit=optimum_circuit, seed=42)
csdm

DensityMatrix([[ 5.40625000e-01+0.00000000e+00j,
                -3.18750000e-01+3.18750000e-02j,
                 2.97500000e-01+1.06250000e-02j,
                -9.56250000e-02+1.33226763e-17j,
                -2.01875000e-01+2.12500000e-02j,
                -2.12500000e-02-9.56250000e-02j,
                 1.48750000e-01+3.18750000e-02j,
                -5.31250000e-02+6.37500000e-02j,
                -1.11022302e-17+3.18750000e-02j,
                -1.16875000e-01-8.50000000e-02j,
                 3.08125000e-01+2.66453526e-17j,
                 1.06250000e-01-5.31250000e-02j,
                -2.12500000e-02+7.43750000e-02j,
                -5.31250000e-02+1.06250000e-01j,
                 3.18750000e-02+6.37500000e-02j,
                 1.06250000e-01+9.56250000e-02j],
               [-3.18750000e-01-3.18750000e-02j,
                 2.21875000e-01+0.00000000e+00j,
                -2.86875000e-01+8.50000000e-02j,
                 1.48750000e-01-3.18750000e-02j,
                 2.

Map density matrix (`dm`) into fermionic operators

1. Convert `dm` to `Operator`
2. Create `SparsePauliOp` from `Operator`
3. Map Pauli `Operator` by using `reverse_jwt`

In [6]:
csdm_op = csdm.to_operator()
csdm_pauli = SparsePauliOp.from_operator(csdm_op)
csdm_fermionic_op = reverse_jwt(csdm_pauli)

csdm_fermionic_op

FermionicOp({'': (0.5406249999999988+0j), '+_0': (-0.31874999999999976-0.03187499999999997j), '-_0': (-0.31874999999999976+0.03187499999999997j), '+_0 -_0': (-0.31875000000000003+0j), '+_0 +_1 -_0': (0.4462499999999997+0.021249999999999908j), '+_1': (0.29749999999999976-0.010625000000000044j), '+_0 -_0 -_1': (-0.4462499999999997+0.021249999999999908j), '-_1': (0.29749999999999976+0.010625000000000044j), '-_0 -_1': (0.0956249999999999-1.0408340855860843e-17j), '+_0 -_1': (-0.2868749999999998+0.08499999999999991j), '+_1 -_0': (-0.2868749999999998-0.08499999999999991j), '+_0 +_1': (-0.0956249999999999-1.0408340855860843e-17j), '+_1 -_1': (-0.4462499999999999+0j), '+_0 +_1 -_1': (0.33999999999999975-0.021250000000000005j), '+_1 -_0 -_1': (-0.33999999999999975-0.021250000000000005j), '+_0 +_1 -_0 -_1': (-0.12750000000000014+0j), '+_0 +_2 -_0': (-0.3399999999999996+0.021249999999999974j), '+_0 -_0 -_2': (0.3399999999999996+0.021249999999999974j), '+_0 +_1 -_0 -_1 -_2': (0.21249999999999972-0

In [7]:
print(f"# of spin orbitals: {csdm_fermionic_op.num_spin_orbitals}\n# of terms {len(csdm_fermionic_op)}")

# of spin orbitals: 4
# of terms 250
